# PolyLogToolkit

*LogToolkit.py* is a python script implemented to support symbolic computations of multiple polylogarithms, detailed in Haoran Li's dissertation [Hopf Algebra of Multiple Polylogarithms and Its Associated One Forms](https://lihaoranicefire.github.io/math/LogToolKit/HopfAlgebraOfMultiplePolylogarithmsAndItsAssociatedOneForms.pdf)

In [2]:
from sympy import *
# import LogToolkit as ltk

In [ ]:
for(int i = 0; i < 1000; i++) {
    print(i)
}

In [ ]:
from sympy import symbols, Add, Mul

# Define a simple class for the wedge product
class WedgeProduct:
    def __init__(self, *components):
        self.components = tuple(components)

    def __repr__(self):
        return f"WedgeProduct{self.components}"

    @staticmethod
    def wedge(*args):
        # Case 1: Single input (like Length[{a}] === 1 in Mathematica)
        if len(args) == 1:
            return args[0]

        # Case 2: Handle sum (S_Plus in Mathematica)
        if isinstance(args[1], Add):
            return sum(WedgeProduct.wedge(*[arg]) for arg in args[1].args)

        # Case 3: Handle zero case (0)
        if any(arg == 0 for arg in args):
            return 0

        # Case 4: Handle multiplication of terms (Times[b___, s_, c___] in Mathematica)
        if isinstance(args[1], Mul):
            # Assume the second argument is a product
            first_term = args[1].args[0]
            return first_term * WedgeProduct.wedge(*args[1].args[1:])

        # Handle generic case for wedge product
        return WedgeProduct(*args)


# Testing the implementation

# Case 1: Single input (returns the input as is)
v1 = WedgeProduct(1, 2, 3)
result_1 = WedgeProduct.wedge(v1)
print(result_1)  # Expected: WedgeProduct(1, 2, 3)

# Case 2: Sum case (S_Plus)
v2 = WedgeProduct(1, 2)
v3 = WedgeProduct(3, 4)
result_2 = WedgeProduct.wedge(v2 + v3)
print(result_2)  # Expected: WedgeProduct(1, 2) + WedgeProduct(3, 4)

# Case 3: Zero case (0)
result_3 = WedgeProduct.wedge(v1 + 0)
print(result_3)  # Expected: 0

# Case 4: Product case (Times[b___, s_, c___])
result_4 = WedgeProduct.wedge(2 * v1 + 3 * v2)
print(result_4)  # Expected: 2 * WedgeProduct(1, 2, 3) + 3 * WedgeProduct(1, 2)